In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, from_json

In [2]:
spark = SparkSession.builder \
    .appName("migrate") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0,org.postgresql:postgresql:42.7.3") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/roy/.ivy2/cache
The jars for the packages stored in: /home/roy/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d0a88e46-56f6-4033-a98a-e192fa471c27;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.0 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 420ms :: artifacts dl 23ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [defa

In [9]:
df = spark.read.format("mongo") \
    .option("uri", "mongodb://192.168.1.16:27017/test.companies") \
    .load()

In [16]:
df.show()

+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+--------------+---------------+--------------------+-------------------+--------------------+-----------+-------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+----------------+--------------------+--------------------+
|                 _id|         acquisition|        acquisitions|        alias_list|       blog_feed_url|            blog_url|  category_code|        competitions|          created_at|      crunc

In [31]:
df_jdbc = df.withColumn("_id", df['_id'].getField("oid").cast("string"))

In [54]:
df.select('_id.*', 'acquisition.*', 'acquisitions').show()

+--------------------+------------+-------------------+--------------+--------------------+--------------------+-------------+--------------+------------+--------------------+--------------------+
|                 oid|price_amount|price_currency_code|     term_code|          source_url|  source_description|acquired_year|acquired_month|acquired_day|   acquiring_company|        acquisitions|
+--------------------+------------+-------------------+--------------+--------------------+--------------------+-------------+--------------+------------+--------------------+--------------------+
|52cdef7c4bab8bd67...|    30000000|                USD|cash_and_stock|http://allthingsd...| Viggle Tries to ...|         2013|            12|          16|    {Viggle, viggle}|                  []|
|52cdef7c4bab8bd67...|        NULL|               NULL|          NULL|                NULL|                NULL|         NULL|          NULL|        NULL|                NULL|                  []|
|52cdef7c4bab8b

In [38]:
df.createOrReplaceTempView("ddf")

In [40]:
spark.sql('select * from ddf').printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- acquisition: struct (nullable = true)
 |    |-- price_amount: long (nullable = true)
 |    |-- price_currency_code: string (nullable = true)
 |    |-- term_code: string (nullable = true)
 |    |-- source_url: string (nullable = true)
 |    |-- source_description: string (nullable = true)
 |    |-- acquired_year: integer (nullable = true)
 |    |-- acquired_month: integer (nullable = true)
 |    |-- acquired_day: integer (nullable = true)
 |    |-- acquiring_company: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- permalink: string (nullable = true)
 |-- acquisitions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- price_amount: double (nullable = true)
 |    |    |-- price_currency_code: string (nullable = true)
 |    |    |-- term_code: string (nullable = true)
 |    |    |-- source_url: string (nullable = true)
 |    |    |-

In [55]:
df_jdbc.schema.fields

[StructField('_id', StringType(), True),
 StructField('acquisition', StructType([StructField('price_amount', LongType(), True), StructField('price_currency_code', StringType(), True), StructField('term_code', StringType(), True), StructField('source_url', StringType(), True), StructField('source_description', StringType(), True), StructField('acquired_year', IntegerType(), True), StructField('acquired_month', IntegerType(), True), StructField('acquired_day', IntegerType(), True), StructField('acquiring_company', StructType([StructField('name', StringType(), True), StructField('permalink', StringType(), True)]), True)]), True),
 StructField('acquisitions', ArrayType(StructType([StructField('price_amount', DoubleType(), True), StructField('price_currency_code', StringType(), True), StructField('term_code', StringType(), True), StructField('source_url', StringType(), True), StructField('source_description', StringType(), True), StructField('acquired_year', IntegerType(), True), StructFiel

In [5]:
df.filter(df['_id']==2).show()

+---+--------------------+----------+----------+--------------------+---------+-------------------+--------------------+-------+--------------------+--------------------+
|_id|             authors|categories|      isbn|     longDescription|pageCount|      publishedDate|    shortDescription| status|        thumbnailUrl|               title|
+---+--------------------+----------+----------+--------------------+---------+-------------------+--------------------+-------+--------------------+--------------------+
|  2|[W. Frank Ableson...|    [Java]|1935182722|When it comes to ...|      592|2011-01-14 03:00:00|Android in Action...|PUBLISH|https://s3.amazon...|Android in Action...|
+---+--------------------+----------+----------+--------------------+---------+-------------------+--------------------+-------+--------------------+--------------------+



In [7]:
## PSOTGRES
server_target = "192.168.1.16:5432"
db_target = "internal"
user_target = "postgres"
password_target = "postgres"
url_target = "jdbc:postgresql://" + server_target + "/" + db_target
prop_target = {
    "user": user_target,
    "password": password_target
}
table_target = "raw.books"

In [8]:
df.write.jdbc(url=url_target, table=table_target, properties=prop_target)